# Clothing 1M data download and preparation

## Data download

In [ ]:
import numpy as np

links = """https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYdFpwZ09fNzF3NjA&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYb3ppZEQ0UnNwUmM&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYTTdOWVdib0x2cVU&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYN2ZvUzNFUkQ5ZGM&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYblNlYXhKVWNYVFU&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYWDAzWkVCWmhqYjg&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYX3lnY2ZIcERVOGM&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYTjZsUkhaU3hzZTA&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYVW5QT0l2U2JJSmc&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYZTlaYnBaNHg4dmM&export=download
https://drive.google.com/u/1/uc?id=0B67_d0rLRTQYUWFyeHVfX0s3QzQ&export=download
"""

start = "https://drive.google.com/u/1/uc?id="
end = "export=download\n"

file_ids = links.replace(start, "").replace(end, "").split("&")[:-1]
file_ids

f = open("download.txt", "r")
target_url = f.read()
target_url

i = 1
file_id = file_ids[i]
print(file_id, "\n\n")
filename = f"{i}.tar"
final_url = target_url.replace("FILE_ID", file_id).replace("FILE_NAME", filename)
print(final_url)

i = 10
if i ==10:
    file_id = file_ids[i]
    filename = f"annotations.tar"
    final_url = target_url.replace("FILE_ID", file_id).replace("FILE_NAME", filename)
    print(final_url)

In [ ]:
import sys
sys.path.append("..")
import numpy as np
import random


import pickle
import time
import itertools
import logging
from tqdm import tqdm
import os
import datetime
import random
import pandas as pd
import models
import shutil
import matplotlib.pyplot as plt
import numpy as npn
import torchvision
from collections import Counter
from sklearn.model_selection import StratifiedKFold
import utils
import warnings
from PIL import Image
import h5py

plt.ion()
plt.show()
%load_ext autoreload
%autoreload 2
from sklearn.exceptions import ConvergenceWarning
with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=ConvergenceWarning)

In [ ]:
path = "../../../datasets/clothing1M"
dataset_name = "clothing1M"

In [ ]:
fn = f"{path}/annotations/category_names_eng.txt"
class_names = {}
with open(fn, 'r') as rf:
    for i, line in enumerate (rf.readlines()):
        class_names[i] = line.rstrip()
class_names

In [ ]:
fn = f"{path}/annotations/clean_label_kv.txt"

df = pd.read_csv(fn, sep=" ", header= None)
print(df.shape)
df.head()

In [ ]:
plt.title("class balance clean data")
df[1].value_counts().plot(kind = "bar")

## Create train, test, val splits

### Create train/val

In [ ]:
skf = StratifiedKFold(n_splits=6,shuffle=True, random_state=1) # generate folds
X = np.arange(df.shape[0])

In [ ]:
for j, (train_index, val_index) in enumerate(skf.split(X, df[1].values)):
    break
print(len(train_index), len(val_index))

In [ ]:
df.iloc[train_index].reset_index(drop=True).to_pickle(f"{path}/annotations/clean_train.pkl")

### Create small test for knn

In [ ]:
df = df.iloc[val_index].reset_index(drop=True)
df.shape

In [ ]:
skf = StratifiedKFold(n_splits=3,shuffle=True, random_state=1) # generate folds
X = np.arange(df.shape[0])

In [ ]:
for j, (val_index, test_index) in enumerate(skf.split(X, df[1].values)):
    break
len(val_index), len(test_index)

In [ ]:
df.iloc[val_index].reset_index(drop=True).to_pickle(f"{path}/annotations/clean_val.pkl")
df.iloc[test_index].reset_index(drop=True).to_pickle(f"{path}/annotations/clean_test.pkl")

## Create h5 files

In [ ]:
for name in [ "test", "val", "train",]:
    df = pd.read_pickle(f"{path}/annotations/clean_{name}.pkl")
    with h5py.File(f"{path}/annotations/clean_{name}.hdf5", 'a') as h:
        # Create dataset inside HDF5 file to store images
        images = h.create_dataset('images', (df.shape[0], img_size, img_size, 3), dtype='uint8')
        print("\nReading  images and captions, storing to file...\n" )

        for i in tqdm(range(df.shape[0])):
            img_path, _ = df.iloc[i].values
            image = Image.open(f"{path}/{img_path}")
            image = image.resize((img_size, img_size), Image.ANTIALIAS)
            images[i] = np.asarray(image)  # Save image to HDF5 file

        labels = df[1].values
        print(f"Min label {min(labels)}, Max {max(labels)}")
        np.save(f"{path}/annotations/clean_{name}.npy", labels)
    h.close()